# Summarize K-Means (K=100) Cluster for full-tfidf

In [2]:
# modify these for your own computer
repo_directory = '/Users/Michael/Documents/GitHub/law-net/'

data_dir = '/Users/Michael/Desktop/network_data/'

import os
import numpy as np
import re
import sys
import matplotlib.pyplot as plt
import glob
import cPickle  as pickle
from collections import OrderedDict
import json


# graph package
import igraph as ig


# stat
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import adjusted_mutual_info_score as ami
from sklearn.metrics import mutual_info_score as mi
from sklearn.metrics import adjusted_rand_score as ar
from sklearn.metrics import calinski_harabaz_score as ch # (X, labels)
from sklearn.metrics import completeness_score as cs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import fowlkes_mallows_score as fm
from sklearn.metrics import homogeneity_completeness_v_measure as hcvm
from sklearn.metrics import homogeneity_score as hs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import silhouette_score as ss # (X, labels)
from sklearn.metrics import silhouette_samples as ss2 # (X, labels)
from sklearn.metrics import v_measure_score as vm

import scipy.sparse
import random
import itertools
from itertools import combinations


# our code
sys.path.append(repo_directory + 'code/')
from summarize_clusters import *
from helpful_functions import *

sys.path.append(repo_directory + 'vertex_metrics_experiment/code/')
from bag_of_words import * 

# which network to download data for
network_name = 'scotus' # 'federal', 'ca1', etc


# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'
nlp_dir = subnet_dir + 'nlp/'
nlp_sub_dir = nlp_dir + 'bow_tfidf/' #tfidf matrix (and other info, i.e. vocab) computed from bag-of-words matrix
nlp_bow_dir = nlp_dir + 'bow/' #bag-of-words matrix (and other info, i.e. vocab)
nlp_df_sub_dir = nlp_dir + 'bow_tfidf_df/'

# csv location
csv_dir = "C:/Users/Michael/Documents/GitHub/law-net/csv/"
csv_dir_mod = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_modularity/"
csv_dir_walk = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_walktrap/"
csv_dir_full_tfidf_km10 = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_full_tfidf_km10/"
csv_dir_full_tfidf_km100 = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_full_tfidf_km100/"

# all the file paths for .txt files
file_paths = glob.glob(text_dir + '*.txt')

# all opinions
all_the_opinions = all_opinions(file_paths)

# clusters directory
clusters_dir = "C:/Users/Michael/Desktop/network_data/raw/scotus/clusters/"

# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## load tf-idf vectors
**tfidf_matrix** = (row_index, column_index): tf_idf value (**CSR FORMAT**)  
**op_id_to_bow_id** = opinion_id (corresponds to row indices)  
**vocab** = all the words in tfidf_matrix (correspond to column indices)

In [3]:
tfidf_matrix, op_id_to_bow_id, vocab = load_tf_idf(nlp_sub_dir)

In [4]:
tfidf_matrix

<27885x567570 sparse matrix of type '<type 'numpy.float64'>'
	with 20817470 stored elements in Compressed Sparse Row format>

In [5]:
clusters = pd.read_csv(csv_dir + 'clusters_full_tfidf.csv')
clusters.head()

,Unnamed: 0,mod,wt,km_10,km_100
0,145658,1,5,8,76
1,89370,3,294,8,76
2,89371,0,35,0,43
3,89372,0,3,8,59
4,89373,0,3,8,11


In [6]:
nlp_tfidf_clusters = clusters['km_100'].tolist()

nlp_clusters = pd.Series(nlp_tfidf_clusters, index=op_id_to_bow_id)
nlp_clusters.head()

145658    76
89370     76
89371     43
89372     59
89373     11
dtype: int64

## get the top 5 biggest clusters

In [7]:
'''
dict_top_n_clusters = dictionary of top K clusters 
                      (key=cluster #, value=opinions in cluster)
                      
biggest_n_clusters = list of top K clusters (int)
'''

dict_top_n_clusters, biggest_n_clusters = get_top_n_clusters(5, 100, nlp_clusters)

cluster 59 : 2012 opinions
cluster 26 : 1092 opinions
cluster 9 : 673 opinions
cluster 13 : 652 opinions
cluster 14 : 609 opinions


# Top K Words of Each Cluster
This function summarizes a set of opinions by returning the words that appear in these opinions with the highest tf-idf scores.

# Top K Words ($\mu_{cluster}$) of Each Cluster
compute the mean tf-idf vector of the cluster, return the top K words from this mean vector

# Top K Words ($\mu_{cluster} - \mu_{complement}$ ) of Each Cluster
compute the mean tf-idf vector of the cluster and also of the complement of the cluster,  
take the difference mu_cluster - mu_complement, return the top K words in this difference

# Most Relevant Opinion of Each Cluster
compute the mean tf-idf vector, return the document in the cluster closet to the mean  

# Cluster 59 Summary (2012 opinions)

In [8]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[59]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_full_tfidf_km100 + "cluster_59.csv")

print cluster_info.shape

(2012, 3)
Wall time: 908 ms


In [9]:
cluster_info = pd.read_csv(csv_dir_full_tfidf_km100 + 'cluster_59.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Barkley v. Levee Commissioners,1876-12-18,https://www.courtlistener.com/opinion/89372/ba...
1,1,Miller v. Life Ins. Co.,1871-11-13,https://www.courtlistener.com/opinion/88391/mi...
2,2,Ward v. Smith,1869-03-29,https://www.courtlistener.com/opinion/88026/wa...
3,3,Payne v. Hook,1869-04-12,https://www.courtlistener.com/opinion/88024/pa...
4,4,United States v. Adams,1869-04-12,https://www.courtlistener.com/opinion/88028/un...


In [10]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[59], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[59], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[59], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[59], tfidf_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_full_tfidf_km100 + "cluster_59_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 20.8 s


In [11]:
cluster_summary = pd.read_csv(csv_dir_full_tfidf_km100 + 'cluster_59_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,carusi,court,interst,92702
1,1,simplex,state,claimant,92702
2,2,gomila,v,creditor,92702
3,3,kountz,case,estat,92702
4,4,schreyer,act,bankruptci,92702


# Cluster 26 Summary (1092 opinions)

In [12]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[26]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_full_tfidf_km100 + "cluster_26.csv")

print cluster_info.shape

(1092, 3)
Wall time: 369 ms


In [13]:
cluster_info = pd.read_csv(csv_dir_full_tfidf_km100 + 'cluster_26.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Santa Anna v. Frank,1885-01-26,https://www.courtlistener.com/opinion/91297/sa...
1,1,Texas & Pacific R. Co. v. Harvey,1913-04-14,https://www.courtlistener.com/opinion/2621063/...
2,2,United States v. Keitel,1908-12-14,https://www.courtlistener.com/opinion/96905/un...
3,3,Norris v. Crocker,1852-05-27,https://www.courtlistener.com/opinion/86752/no...
4,4,Norton Co. v. Department of Revenue of Ill.,1951-05-21,https://www.courtlistener.com/opinion/104862/n...


In [14]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[26], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[26], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[26], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[26], tfidf_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_full_tfidf_km100 + "cluster_26_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 12.6 s


In [15]:
cluster_summary = pd.read_csv(csv_dir_full_tfidf_km100 + 'cluster_26_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,shawhan,court,tax,106447
1,1,reinhard,state,court,106447
2,2,metsker,v,petition,106447
3,3,kaupp,case,appeal,106447
4,4,crudup,act,circuit,106447


# Cluster 9 Summary (673 opinions)

In [16]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[9]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_full_tfidf_km100 + "cluster_9.csv")

print cluster_info.shape

(673, 3)
Wall time: 304 ms


In [17]:
cluster_info = pd.read_csv(csv_dir_full_tfidf_km100 + 'cluster_9.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Ratterman v. Western Union Telegraph Co.,1888-05-14,https://www.courtlistener.com/opinion/92269/ra...
1,1,Miller v. Fenton,1985-12-03,https://www.courtlistener.com/opinion/111542/m...
2,2,Lake Coal Co. v. Roberts & Schaeffer Co.,1985-12-03,https://www.courtlistener.com/opinion/111543/l...
3,3,DE Foote & Co. v. Stanley,1914-02-24,https://www.courtlistener.com/opinion/98107/de...
4,4,Davis v. Federal Election Com'n,2008-06-26,https://www.courtlistener.com/opinion/145776/d...


In [18]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[9], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[9], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[9], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[9], tfidf_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_full_tfidf_km100 + "cluster_9_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 8.8 s


In [19]:
cluster_summary = pd.read_csv(csv_dir_full_tfidf_km100 + 'cluster_9_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,dispensari,court,bank,90667
1,1,gumaer,state,tax,90667
2,2,juvenil,v,indict,90667
3,3,ch,case,conspiraci,90667
4,4,wesenberg,act,receiv,90667


# Cluster 13 Summary (652 opinions)

In [20]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[13]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_full_tfidf_km100 + "cluster_13.csv")

print cluster_info.shape

(652, 3)
Wall time: 192 ms


In [21]:
cluster_info = pd.read_csv(csv_dir_full_tfidf_km100 + 'cluster_13.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Thurston v. United States,1914-02-24,https://www.courtlistener.com/opinion/98103/th...
1,1,The Adula,1900-02-26,https://www.courtlistener.com/opinion/95188/th...
2,2,Southern R. Co. v. Carnegie Steel Co.,1900-01-29,https://www.courtlistener.com/opinion/95181/so...
3,3,County of Cass v. Johnston,1877-11-12,https://www.courtlistener.com/opinion/89611/co...
4,4,Inter-Island Steam Nav. Co. v. Byrne,1915-12-20,https://www.courtlistener.com/opinion/98600/in...


In [22]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[13], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[13], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[13], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[13], tfidf_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_full_tfidf_km100 + "cluster_13_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 9.16 s


In [24]:
cluster_summary = pd.read_csv(csv_dir_full_tfidf_km100 + 'cluster_13_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,wool,court,juri,106170
1,1,tsc,state,habea,106170
2,2,burner,v,sentenc,106170
3,3,bendey,case,instruct,106170
4,4,aldrich,act,employ,106170


# Cluster 14 Summary (609 opinions)

In [25]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[14]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_full_tfidf_km100 + "cluster_14.csv")

print cluster_info.shape

(609, 3)
Wall time: 212 ms


In [26]:
cluster_info = pd.read_csv(csv_dir_full_tfidf_km100 + 'cluster_14.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Oklahoma Tax Comm'n v. Graham,1989-03-29,https://www.courtlistener.com/opinion/112230/o...
1,1,Finlay v. King's Lessee,1830-02-18,https://www.courtlistener.com/opinion/85678/fi...
2,2,Boylan v. Hot Springs R. Co.,1889-11-11,https://www.courtlistener.com/opinion/92579/bo...
3,3,Graham County Soil & Water Conservation Dist. ...,2005-06-20,https://www.courtlistener.com/opinion/799981/g...
4,4,Lowe v. SEC,1985-06-10,https://www.courtlistener.com/opinion/111467/l...


In [27]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[14], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[14], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[14], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[14], tfidf_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_full_tfidf_km100 + "cluster_14_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 8.79 s


In [28]:
cluster_summary = pd.read_csv(csv_dir_full_tfidf_km100 + 'cluster_14_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,ree,court,patent,90041
1,1,tea,state,street,90041
2,2,frog,v,assigne,90041
3,3,dagg,case,juri,90041
4,4,renick,act,invent,90041
